In [1]:
import numpy as np
import arepo
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit

In [2]:
sim_list = ['Nbody', 'phantom-vacuum-Sg20-Rc3.5']

basepath = '/n/holystore01/LABS/hernquist_lab/Users/abeane/starbar_runs/runs/'

def read_snap(idx, sim_idx, lvl='lvl3', parttype=[0], fields=['Coordinates', 'Masses', 'Velocities', 'ParticleIDs', 'Potential'],
              basepath = basepath):
    fname = basepath + sim_list[sim_idx] + '/' + lvl + '/output'
    return arepo.Snapshot(fname, idx, parttype=parttype, fields=fields, combineFiles=True)

def read_sfr(sim_idx, lvl='lvl3', basepath = basepath):
    fname = basepath + sim_list[sim_idx] + '/' + lvl + '/output/sfr.txt'
    return np.genfromtxt(fname)

def read_fourier(sim_idx, lvl='lvl3', basepath='/n/home01/abeane/starbar/plots/'):
    sim = sim_list[sim_idx]
    f = h5.File(basepath+'/fourier_component/data/fourier_'+sim+'-'+lvl+'.hdf5', mode='r')
    return f

In [16]:
sn = read_snap(300, 0, parttype=None, fields=None)

In [25]:
def get_Menc_ptype(sn, ptype, rcut=4):
    if sn.NumPart_Total[ptype]==0:
        return 0.0
    
    part = getattr(sn, 'part'+str(ptype))
    
    # center = np.array([sn.BoxSize, sn.BoxSize, sn.BoxSize])/2.
    center = sn.part1.pos.value[np.argmin(sn.part1.pot.value)]
    
    pos = part.pos.value - center
    r = np.linalg.norm(pos, axis=1)
    key = r < rcut
    
    if sn.MassTable[ptype] > 0:
        Menc = len(np.where(key)[0]) * sn.MassTable[ptype]
    else:
        Menc = np.sum(part.mass.value[key])
    
    return Menc

In [26]:
Menc_0 = get_Menc_ptype(sn, 0)
Menc_1 = get_Menc_ptype(sn, 1)
Menc_2 = get_Menc_ptype(sn, 2)
Menc_3 = get_Menc_ptype(sn, 3)
Menc_4 = get_Menc_ptype(sn, 4)

In [27]:
Menc_DM = Menc_1
Menc_bar = Menc_0 + Menc_2 + Menc_3 + Menc_4

In [28]:
fDM = Menc_DM / (Menc_bar + Menc_DM)

In [29]:
fDM

0.38780556 

In [24]:
Menc_2

0. 1e+10 Msol